In [2]:
import pandas as pd
import re
import os
import pytz
from datetime import datetime

In [2]:
#### THIS IS THE ONLY CELL CONTAINING ELEMENTS THAT NEED TO BE CHANGED
    # 1) the directory containing leg 1 underway data
    # 2) the directory containing leg 2 underway data
    # 3) the cruise number - this is just used for the csv name saved at the end of the notebook

#the directory containing the csv files from the underway system
##each file is a day, and the file is data collected every minute
dir_leg_1 = "/AR82-Pioneer20/AR82_armstrong_underway_data/leg1/proc" # change me (1)
dir_leg_2 = "/AR82-Pioneer20/AR82_armstrong_underway_data/leg2/proc" # change me (2)
cruise_name = "AR82" # change me (3) 

In [4]:
#the regex pattern that will match the correct underway files
file_name_pattern = r'AR\d*_0000.csv'
#compile the regex pattern
pattern_regex = re.compile(file_name_pattern)

In [5]:
#get a list of all csv files in the directory matching the pattern above
leg_1_csv_files = [file for file in os.listdir(dir_leg_1) if pattern_regex.match(file)]
leg_2_csv_files = [file for file in os.listdir(dir_leg_2) if pattern_regex.match(file)]

#empty list to store individual dataframes before merge
dfs = []

#iterate over each csv file, read as df, and append to dfs list
for file in leg_1_csv_files:
    file_path = os.path.join(dir_leg_1, file)
    df = pd.read_csv(file_path, header=1)
    dfs.append(df)
    
for file in leg_2_csv_files:
    file_path = os.path.join(dir_leg_2, file)
    df = pd.read_csv(file_path, header=1)
    dfs.append(df)

#concatenate all dfs in the list into a single df
merged_underway_df = pd.concat(dfs, ignore_index=True)

In [6]:
merged_underway_df

,DATE_GMT,TIME_GMT,Dec_LAT,Dec_LON,SPD,HDT,DPS_COG,SOG,WXTP_Ta,WXTS_Ta,...,SBE45S,SBE48T,FLR,FLOW,SSVdslog,Depth12,Depth35,EM122,EM710,COG
0,2024/04/03,00:00:47.688,37.362,-74.208,10.84,207.11,207.24,11.08,11.1,11.1,...,32.6663,8.5424,365.4,45.4,1481.711,NAN,NAN,NAN,NAN,NaN
1,2024/04/03,00:01:47.688,37.359,-74.21,10.89,207.3,205.66,11.05,11.0,11.0,...,32.666,8.5465,355.3,45.4,1481.726,NAN,NAN,NAN,NAN,NaN
2,2024/04/03,00:02:47.688,37.356,-74.212,10.89,208.64,207.46,11.27,11.0,11.0,...,32.6633,8.5821,352.3,45.2,1481.856,NAN,NAN,NAN,NAN,NaN
3,2024/04/03,00:03:47.688,37.353,-74.214,10.94,208.7,207.94,11.25,11.0,11.1,...,32.6614,8.6179,347.6,45.3,1481.988,NAN,NAN,NAN,NAN,NaN
4,2024/04/03,00:04:47.688,37.351,-74.215,10.9,208.27,205.49,11.32,11.0,11.2,...,32.6601,8.6154,340.8,45.3,1481.977,NAN,NAN,NAN,NAN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31518,2024/04/16,23:55:48.965,35.943,-74.831,NAN,265.85,269.53,6.92,13.6,13.6,...,31.7598,11.95,480.0,60.2,1492.866,NAN,NAN,NAN,NAN,NaN
31519,2024/04/16,23:56:48.965,35.943,-74.834,NAN,265.7,268.66,6.91,13.6,13.7,...,31.7605,11.93,479.3,60.0,1492.798,NAN,NAN,NAN,NAN,NaN
31520,2024/04/16,23:57:48.965,35.943,-74.836,NAN,265.65,268.40,7.07,13.6,13.7,...,31.7625,11.9123,480.0,60.1,1492.739,NAN,NAN,NAN,NAN,NaN
31521,2024/04/16,23:58:48.965,35.943,-74.838,NAN,265.76,279.04,6.98,13.6,13.7,...,31.7649,11.8922,481.2,60.2,1492.673,NAN,NAN,NAN,NAN,NaN


In [7]:
# create a datetime field to match with the IFCB datetimes
merged_underway_df['Datetime_GMT_underway'] = merged_underway_df['DATE_GMT'].str.cat(merged_underway_df[' TIME_GMT'], sep = "")
merged_underway_df

,DATE_GMT,TIME_GMT,Dec_LAT,Dec_LON,SPD,HDT,DPS_COG,SOG,WXTP_Ta,WXTS_Ta,...,SBE48T,FLR,FLOW,SSVdslog,Depth12,Depth35,EM122,EM710,COG,Datetime_GMT_underway
0,2024/04/03,00:00:47.688,37.362,-74.208,10.84,207.11,207.24,11.08,11.1,11.1,...,8.5424,365.4,45.4,1481.711,NAN,NAN,NAN,NAN,NaN,2024/04/03 00:00:47.688
1,2024/04/03,00:01:47.688,37.359,-74.21,10.89,207.3,205.66,11.05,11.0,11.0,...,8.5465,355.3,45.4,1481.726,NAN,NAN,NAN,NAN,NaN,2024/04/03 00:01:47.688
2,2024/04/03,00:02:47.688,37.356,-74.212,10.89,208.64,207.46,11.27,11.0,11.0,...,8.5821,352.3,45.2,1481.856,NAN,NAN,NAN,NAN,NaN,2024/04/03 00:02:47.688
3,2024/04/03,00:03:47.688,37.353,-74.214,10.94,208.7,207.94,11.25,11.0,11.1,...,8.6179,347.6,45.3,1481.988,NAN,NAN,NAN,NAN,NaN,2024/04/03 00:03:47.688
4,2024/04/03,00:04:47.688,37.351,-74.215,10.9,208.27,205.49,11.32,11.0,11.2,...,8.6154,340.8,45.3,1481.977,NAN,NAN,NAN,NAN,NaN,2024/04/03 00:04:47.688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31518,2024/04/16,23:55:48.965,35.943,-74.831,NAN,265.85,269.53,6.92,13.6,13.6,...,11.95,480.0,60.2,1492.866,NAN,NAN,NAN,NAN,NaN,2024/04/16 23:55:48.965
31519,2024/04/16,23:56:48.965,35.943,-74.834,NAN,265.7,268.66,6.91,13.6,13.7,...,11.93,479.3,60.0,1492.798,NAN,NAN,NAN,NAN,NaN,2024/04/16 23:56:48.965
31520,2024/04/16,23:57:48.965,35.943,-74.836,NAN,265.65,268.40,7.07,13.6,13.7,...,11.9123,480.0,60.1,1492.739,NAN,NAN,NAN,NAN,NaN,2024/04/16 23:57:48.965
31521,2024/04/16,23:58:48.965,35.943,-74.838,NAN,265.76,279.04,6.98,13.6,13.7,...,11.8922,481.2,60.2,1492.673,NAN,NAN,NAN,NAN,NaN,2024/04/16 23:58:48.965


In [8]:
# convert the datetime format
def gmt_to_utc(gmt_datetime_str):
    gmt = pytz.timezone('GMT')
    datetime_formats = [
        '%Y/%m/%d %H:%M:%S',
        '%Y-%m-%d %H:%M:%S',
        '%Y/%m/%d %H:%M:%S.%f'
    ]

    for fmt in datetime_formats:
        try:
            gmt_datetime = datetime.strptime(gmt_datetime_str, fmt)
            gmt_localized_datetime = gmt.localize(gmt_datetime)
            utc_datetime = gmt_localized_datetime.astimezone(pytz.utc)
            return utc_datetime
        except ValueError:
            continue
    raise ValueError(f"Time data '{gmt_datetime_str}' does not match any known format")

merged_underway_df['Datetime_UTC_underway'] = merged_underway_df['Datetime_GMT_underway'].apply(gmt_to_utc)
merged_underway_df

,DATE_GMT,TIME_GMT,Dec_LAT,Dec_LON,SPD,HDT,DPS_COG,SOG,WXTP_Ta,WXTS_Ta,...,FLR,FLOW,SSVdslog,Depth12,Depth35,EM122,EM710,COG,Datetime_GMT_underway,Datetime_UTC_underway
0,2024/04/03,00:00:47.688,37.362,-74.208,10.84,207.11,207.24,11.08,11.1,11.1,...,365.4,45.4,1481.711,NAN,NAN,NAN,NAN,NaN,2024/04/03 00:00:47.688,2024-04-03 00:00:47.688000+00:00
1,2024/04/03,00:01:47.688,37.359,-74.21,10.89,207.3,205.66,11.05,11.0,11.0,...,355.3,45.4,1481.726,NAN,NAN,NAN,NAN,NaN,2024/04/03 00:01:47.688,2024-04-03 00:01:47.688000+00:00
2,2024/04/03,00:02:47.688,37.356,-74.212,10.89,208.64,207.46,11.27,11.0,11.0,...,352.3,45.2,1481.856,NAN,NAN,NAN,NAN,NaN,2024/04/03 00:02:47.688,2024-04-03 00:02:47.688000+00:00
3,2024/04/03,00:03:47.688,37.353,-74.214,10.94,208.7,207.94,11.25,11.0,11.1,...,347.6,45.3,1481.988,NAN,NAN,NAN,NAN,NaN,2024/04/03 00:03:47.688,2024-04-03 00:03:47.688000+00:00
4,2024/04/03,00:04:47.688,37.351,-74.215,10.9,208.27,205.49,11.32,11.0,11.2,...,340.8,45.3,1481.977,NAN,NAN,NAN,NAN,NaN,2024/04/03 00:04:47.688,2024-04-03 00:04:47.688000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31518,2024/04/16,23:55:48.965,35.943,-74.831,NAN,265.85,269.53,6.92,13.6,13.6,...,480.0,60.2,1492.866,NAN,NAN,NAN,NAN,NaN,2024/04/16 23:55:48.965,2024-04-16 23:55:48.965000+00:00
31519,2024/04/16,23:56:48.965,35.943,-74.834,NAN,265.7,268.66,6.91,13.6,13.7,...,479.3,60.0,1492.798,NAN,NAN,NAN,NAN,NaN,2024/04/16 23:56:48.965,2024-04-16 23:56:48.965000+00:00
31520,2024/04/16,23:57:48.965,35.943,-74.836,NAN,265.65,268.40,7.07,13.6,13.7,...,480.0,60.1,1492.739,NAN,NAN,NAN,NAN,NaN,2024/04/16 23:57:48.965,2024-04-16 23:57:48.965000+00:00
31521,2024/04/16,23:58:48.965,35.943,-74.838,NAN,265.76,279.04,6.98,13.6,13.7,...,481.2,60.2,1492.673,NAN,NAN,NAN,NAN,NaN,2024/04/16 23:58:48.965,2024-04-16 23:58:48.965000+00:00


In [10]:
# save the file

merged_underway_df.to_csv(f"Merged_Ship_Underway_files/mergedunderway_{cruise_name}.csv", index=False)